In [1]:
import cv2
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import rospy

import pyrealsense2
from realsense_depth import *
import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
#import re





image = None

rospy.init_node("my_pic", anonymous=True)
bridge = CvBridge()
loop_rate = rospy.Rate(0.5) # Node cycle rate (in Hz).

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def grab_frame():
    
    frame_color=rospy.wait_for_message('/camera/color/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_color = bridge.imgmsg_to_cv2(frame_color, desired_encoding='rgb8')
    
    frame_depth = rospy.wait_for_message('/camera/depth/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_depth = bridge.imgmsg_to_cv2(frame_depth)
    
    return cv_image_color, cv_image_depth

In [3]:
def visualize_rgbd(rgbd_image):
    print(rgbd_image)
    o3d.visualization.draw_geometries([rgbd_image])
    
    #intrinsic = o3d.camera.PinholeCameraIntrinsic()
    #intrinsic.intrinsic_matrix =  [[462.1379699707031, 0.0, 320.0], [0.0, 462.1379699707031, 240.0], [0.0, 0.0, 1.0]]
    #intrinsic.intrinsic_matrix =  [[347.99755859375, 0.0, 320.0], [0.0, 347.99755859375, 240.0], [0.0, 0.0, 1.0]]
    #intrinsic.intrinsic_matrix =  [[602.71783447, 0.0, 313.06835938], [0.0, 601.61364746, 230.37461853], [0.0, 0.0, 1.0]]
    
    #w = 640
    #h = 480
    #fx = 602.71783447
    #fy = 601.61364746
    #cx = 313.06835938
    #cy = 230.37461853
    
    #Color frame ROS camera
    w = 640
    h = 480
    fx = 462.1379699707031
    fy = 462.1379699707031
    cx = 320.0
    cy = 240.0
    
    #Depth frame ROS camera
    #w = 640
    #h = 480
    #fx = 347.99755859375
    #fy = 347.99755859375
    #cx = 320.0
    #cy = 240.0    
    
    
    intrinsic = o3d.camera.PinholeCameraIntrinsic(w, h, fx,fy, cx, cy)
    intrinsic.intrinsic_matrix = [[fx, 0, cx], [0, fy, cy], [0, 0, 1]]
    
    cam = o3d.camera.PinholeCameraParameters()
    cam.intrinsic = intrinsic
    
    #cam.extrinsic = np.array([[0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 0.], [0., 0., 0., 1.]])
    #pcd = o3d.geometry.create_point_cloud_from_rgbd_image(rgbd_image, cam.intrinsic, cam.extrinsic)
    
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic)
    #pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,intrinsic)
    
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down.
    
    o3d.visualization.draw_geometries([pcd])
    
    return pcd
    
    
def tst_dataset(color_frame,depth_frame):
    color_raw = o3d.geometry.Image(np.asarray(color_frame))
    depth_raw = o3d.geometry.Image(np.asarray(depth_frame.astype(np.uint16)) )
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity=False)
    pcd = visualize_rgbd(rgbd_image)
    return pcd


def box_pos(x_coord, y_coord, width, height, centered=0):
    if centered == 0:
        start_point = (x_coord, y_coord) # represents the top left corner of rectangle
        end_point = (x_coord+width-1, y_coord+height-1)  # represents the bottom right corner of rectangle
    elif centered == 1:
        new_x = x_coord - (np.floor(width/2)-1).astype(int)
        new_y = y_coord - (np.floor(height/2)-1).astype(int)
        start_point = (new_x, new_y) 
        end_point = (new_x+width, new_y+height)
    return start_point, end_point



In [4]:
color_frame, depth_frame = grab_frame()
print("color frame:",color_frame.shape, " Depth frame:",depth_frame.shape)

#Check center pixel distance

point = (320, 240)

#color_frame, depth_frame = grab_frame()

distance = depth_frame[point[1], point[0]]
print("Center of image:", distance, "mm")

color frame: (480, 640, 3)  Depth frame: (480, 640)
Center of image: 291 mm



start_point,end_point = box_pos(320, 240, 10, 10, centered=1)  #x,y,width,height, 0-- top left coord, 1--- center coord

cf = color_frame
for i in range (color_frame.shape[0]):
    for j in range (color_frame.shape[1]):
        if depth_frame[i][j] == 0:
            cf[i][j] = [0 , 0, 0] 

plt.imshow(cf)
plt.show()
window_name = 'Filtered_image'  # Window name in which image is displayed

cv2.rectangle(cf, start_point, end_point, (0, 0, 255), 1)
cv2.imshow(window_name, cf)
cv2.waitKey(0)
cv2.destroyAllWindows()


#color_frame_crop = cf[start_point[1]:end_point[1], start_point[0]:end_point[0]]

color_frame_crop = cf[start_point[1]:end_point[1], start_point[0]:end_point[0]]
depth_frame_crop = depth_frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

print(depth_frame_crop.shape)
zz=depth_frame_crop.T
print(zz.shape)

#cv2.imshow("cropped", depth_frame_crop)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [5]:
pt_cloud = tst_dataset(color_frame, depth_frame)


RGBDImage of size 
Color image : 640x480, with 3 channels.
Depth image : 640x480, with 1 channels.
Use numpy.asarray to access buffer data.


In [21]:
downpcd = pt_cloud.voxel_down_sample(voxel_size=0.2)
#o3d.visualization.draw_geometries([downpcd])

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30)) #radius in meters

#o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

v1_normal = np.array(np.asarray(downpcd.normals))
v1_normal

Recompute the normal of the downsampled point cloud


array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [40]:
print("Align normals towards camera") #orient_normals_towards_camera_location(self, camera_location=array([0.0, 0.0, 0.0]))

downpcd.orient_normals_towards_camera_location()

o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

v2_camera = np.array(np.asarray(downpcd.normals))
v2_camera

Align normals towards camera


array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [32]:
rec = np.array(downpcd.points)

In [33]:
rec



array([[ 0.09714215, -0.22224315, -0.43535662],
       [-0.26622851,  0.14754292, -0.45154966],
       [-0.07836239,  0.10525681, -0.28782354],
       [ 0.06675173, -0.14658677, -0.29903121],
       [-0.26781304, -0.03859936, -0.45614777],
       [-0.21766147,  0.15068299, -0.45154173],
       [-0.07856745, -0.14345716, -0.294     ],
       [ 0.07195043,  0.10531638, -0.28800668],
       [ 0.28411573,  0.1474998 , -0.45226581],
       [ 0.28165801, -0.19269493, -0.46075358],
       [ 0.07182835, -0.03820167, -0.29156583],
       [ 0.28570682, -0.03895756, -0.45690802],
       [-0.21763339, -0.03689262, -0.45615721],
       [-0.26904887, -0.18872241, -0.45989389],
       [-0.07850997, -0.03810011, -0.29135889],
       [-0.13769084, -0.22291574, -0.45340192]])

In [ ]:
[0.303, 0.017, 0.456]

In [37]:
#zz=downpcd.select_by_index([2,3,6,7,10,14], invert=False)
zz=downpcd.select_by_index([10,14], invert=False)
o3d.visualization.draw_geometries([zz], point_show_normal=True)
nor = np.array(zz.normals)
pts = np.array(zz.points)

In [38]:
nor

array([[0., 0., 1.],
       [0., 0., 1.]])

In [39]:
pts

array([[ 0.07182835, -0.03820167, -0.29156583],
       [-0.07850997, -0.03810011, -0.29135889]])

In [ ]:
print("angle between Normal and Camera")
print()
for i in range(len(v1_normal)):
    print(i+1," ", np.round(np.degrees(angle_between(v1_normal[i], v2_camera[i]))) )

print("Align normals towards tangent plane") 

downpcd.orient_normals_consistent_tangent_plane(10)
#o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

v3_tangent = np.array(np.asarray(downpcd.normals))
v3_tangent

In [ ]:
#print("angle between Normal and Camera,   Normal and Tangent")
#print()
#for i in range(len(v1_normal)):
#    print(i+1," ", np.round(np.degrees(angle_between(v1_normal[i], v2_camera[i]))),"                            ", 
#               np.round(np.degrees(angle_between(v1_normal[i], v3_tangent[i]))) )

In [ ]:
zz1 = downpcd.detect_planar_patches(normal_variance_threshold_deg=60, coplanarity_deg=75, outlier_ratio=0.75, 
                                   min_plane_edge_length=0.0, min_num_points=0, 
                                   search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
zz1

In [ ]:
zz2 = downpcd.detect_planar_patches(normal_variance_threshold_deg=60, coplanarity_deg=75, outlier_ratio=0.75, 
                                   min_plane_edge_length=0.0, min_num_points=0, 
                                   search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
zz2

In [ ]:
print(np.degrees(angle_between(zz1[0].center, zz2[0].center)))
print(np.degrees(angle_between(zz1[1].center, zz2[1].center)))

In [ ]:
#color_frame, depth_frame = grab_frame()
df = np.asarray(depth_frame).flatten()
hi=np.histogram(df,bins=100)
#print(hi[0].shape)
#print(hi[1].shape)

#plt.hist(df,bins=1800)
#plt.title("Distance Distribution of depth Image")
#plt.show()
unique, counts = np.unique(df, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
plt.imshow(depth_frame)
plt.colorbar()
plt.show()

In [ ]:
def box_pos(x_coord, y_coord, width, height, centered=0):
    if centered == 0:
        start_point = (x_coord, y_coord) # represents the top left corner of rectangle
        end_point = (x_coord+width-1, y_coord+height-1)  # represents the bottom right corner of rectangle
    elif centered == 1:
        new_x = x_coord - (np.floor(width/2)-1).astype(int)
        new_y = y_coord - (np.floor(height/2)-1).astype(int)
        start_point = (new_x, new_y) 
        end_point = (new_x+width, new_y+height)
    return start_point, end_point

start_point,end_point = box_pos(320, 240, 300, 300, centered=1)  #x,y,width,height, 0-- top left coord, 1--- center coord

cf = color_frame
for i in range (color_frame.shape[0]):
    for j in range (color_frame.shape[1]):
        if depth_frame[i][j] == 0:
            cf[i][j] = [0 , 0, 0] 

plt.imshow(cf)
plt.show()
window_name = 'Filtered_image'  # Window name in which image is displayed

cv2.rectangle(cf, start_point, end_point, (0, 0, 255), 1)
cv2.imshow(window_name, cf)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
color_crop_img = cf[start_point[1]:end_point[1], start_point[0]:end_point[0]]
crop_img = depth_frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

print(crop_img.shape)
zz=crop_img.T
print(zz.shape)

#cv2.imshow("cropped", crop_img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
li_FB=[]
for i in range(crop_img.shape[0]):
    ci = crop_img[i][crop_img[i]!=0] #filter out 0 values (mostly errors)
    if len(ci)!=0:
        li_FB.append(round(np.mean(ci))) 
print(li_FB)
print()

li_LR=[]
for i in range(crop_img.shape[1]):
    ci = zz[i][zz[i]!=0] #filter out 0 values (mostly errors)
    if len(ci)!=0:
        li_LR.append(round(np.mean(ci))) 
print(li_LR)


In [ ]:
def filter_res(dat_list, max_steps_blocked, outlier_threshold):
    dat_list=np.asarray(dat_list)
    unique, counts = np.unique(dat_list, return_counts=True)
    #print(np.asarray((unique, counts)).T)
    if len(unique) < max_steps_blocked:
        for i in range(len(counts)):
            if counts[i] < outlier_threshold: #if less than threshold, then filter out the result
                dat_list = np.delete(dat_list, np.where(dat_list == unique[i]))
    return dat_list



#filter_res(li_FB,10,10) #list, max blocked steps(adjust this if window size is reduced from 300x300),
                     #outlier threshold,if more than this value, then its no longer outlier


In [ ]:
thresh = 0
#data_FB = np.asarray(li_FB)
#data_LR = np.asarray(li)
data_FB = filter_res(li_FB,10,10)
data_LR = filter_res(li_LR,10,10)

Diff_FB = data_FB[0] - data_FB[-1]
Diff_LR = data_LR[0] - data_LR[-1]

print("Pitch (Front Back)")
print(data_FB[0], data_FB[-1])
print("Difference FB:", Diff_FB)
print()

print("Roll (Left Right)")
print(data_LR[0], data_LR[-1])
print("Difference LR:", Diff_LR)
print()
print()

if abs(Diff_FB) < 10 and abs(Diff_LR) < 10:
    print("Surface is Flat!!")

elif abs(Diff_FB) < 10 and Diff_LR > thresh:
    print("Surface is tilting towards Left")
elif abs(Diff_FB) < 10 and Diff_LR < thresh:
    print("Surface is tilting towards Right")

elif abs(Diff_LR) < 10 and Diff_FB > thresh:
    print("Surface is tilting towards Front")
elif abs(Diff_LR) < 10 and Diff_FB < thresh:
    print("Surface is tilting towards Back")

elif Diff_FB > thresh and Diff_LR < thresh:
    print("Surface is tilting towards Front right")
elif Diff_FB < thresh and Diff_LR < thresh:
    print("Surface is tilting towards Back right")
elif Diff_FB > thresh and Diff_LR > thresh:
    print("Surface is tilting towards Front left")
elif Diff_FB < thresh and Diff_LR > thresh:
    print("Surface is tilting towards Back left")

In [ ]:
#Pitch (Front Back)
idx=np.floor(len(data_FB)/2).astype(int)
if len(data_FB)%2 == 1:
    xx_fb=np.arange(-idx,idx+1)
else:
    xx_fb=np.arange(-idx,idx)

#Roll (Left Right)
idx=np.floor(len(data_LR)/2).astype(int)
if len(data_LR)%2 == 1:
    xx_lr=np.arange(-idx,idx+1)
else:
    xx_lr=np.arange(-idx,idx)    


fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(14, 4))
ax1.set_title('centered data Roll (L-R)')
ax1.plot(xx_lr, data_LR,"ob") 

ax2.set_title('centered data Pitch (F-B)')
a=ax2.plot(xx_fb, data_FB,"ob")


In [ ]:
#mn = np.mean(crop_img[crop_img!=0])
#print(mn)